# Pdf Chatbot Using CrewAI and LlamaIndex

### Install required dependencies

In [1]:
!pip install llama-index --quiet
!pip install llama-index-llms-groq --quiet
!pip install llama-index-core --quiet
!pip install llama-index-readers-file --quiet
!pip install llama-index-tools-wolfram-alpha --quiet
!pip install llama-index-embeddings-huggingface --quiet
!pip install 'crewai[tools]' --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.2/853.2 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3

### Let's setup the LLM with Groq. Sign up at Groq, it is free. Go to Groq console and get the API keys.

In [2]:
from llama_index.llms.groq import Groq

In [3]:
llm = Groq(model="llama3-70b-8192", api_key="your Groq api key")

In [4]:
response = llm.complete("Explain the importance of LLMs")
print(response)

Large Language Models (LLMs) have revolutionized the field of Natural Language Processing (NLP) and have numerous applications across various industries. The importance of LLMs can be summarized as follows:

1. **Improved Language Understanding**: LLMs have demonstrated remarkable capabilities in understanding human language, including nuances, context, and subtleties. This enables them to generate more accurate and informative responses, making them valuable for applications like chatbots, virtual assistants, and language translation.
2. **Enhanced Text Generation**: LLMs can generate high-quality, coherent, and context-specific text, which has numerous applications in content creation, writing assistance, and automated reporting.
3. **Advancements in AI Research**: LLMs have pushed the boundaries of AI research, enabling the development of more sophisticated models and techniques. They have also sparked new areas of research, such as multimodal language processing and explainability.

### Crew AI requires a chat based model for binding

In [5]:
from langchain_openai import ChatOpenAI

# Define your API key here
groq_api_key = "your Groq api key"

chat_llm = ChatOpenAI(
    openai_api_base="https://api.groq.com/openai/v1",
    openai_api_key=groq_api_key,
    model="llama3-70b-8192",
    temperature=0,
    max_tokens=1000,
)

### Download data

### Parse the content of the data

In [6]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.openai import OpenAI
import os
from langchain_openai import ChatOpenAI

In [60]:
reader = SimpleDirectoryReader(input_files=["Morocco-1-3.pdf"])
docs = reader.load_data()
docs[0]

Document(id_='d4591696-908f-4521-864b-c80bbe71c5f2', embedding=None, metadata={'page_label': '1', 'file_name': 'Morocco-1-3.pdf', 'file_path': 'Morocco-1-3.pdf', 'file_type': 'application/pdf', 'file_size': 326422, 'creation_date': '2024-07-29', 'last_modified_date': '2024-07-29'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Kingdom of Morocco\nاﻟﻤﻐﺮﺑﻴﺔ  اﻟﻤﻤﻠﻜﺔ  (Arabic)\nal-Mamlakah al-Maghribiyah\nⵜ ⴰ ⴳ ⵍ ⴷ ⵉ ⵜ  ⵏ  ⵍ ⵎ ⵖ ⵔ ⵉ ⴱ (Tamazight)\nTageldit n Lme ɣ rib\nFlag\n Coat of arms\nMotto: \n"Allāh, al-Wa ṭ an, al-Malik"\n"God, Country, King"[1]\nAnthem: \n"an-Našīd al-Wa ṭ anīy"\n"Cherifian Anthem"\nCapital Rabat\n34°02′N 6°51′W\nLargest city Casablanca\n33°32′N 7°35′W\nMorocco\nMorocco,[d] officially the Kingdom of\nMorocco,[e] is a country in th

### Setup the embedding model

In [61]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [12]:
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

### Build Index

In [62]:
index = VectorStoreIndex.from_documents(docs,
                                        embed_model=embed_model,
                                        )

### Build the Query Engine

In [63]:
query_engine = index.as_query_engine(similarity_top_k=5, llm=llm)

### Instantiate the query engine as a Tool

In [65]:
from crewai_tools import LlamaIndexTool

document_analysis_tool = LlamaIndexTool.from_query_engine(
    query_engine,
    name="Document Analysis Tool",
    description="Use this tool to analyze documents and extract relevant information",
)


In [66]:
query_tool.args_schema.schema()

{'title': 'QueryToolSchema',
 'description': 'Schema for query tool.',
 'type': 'object',
 'properties': {'query': {'title': 'Query',
   'description': 'Search query for the query tool.',
   'type': 'string'}},
 'required': ['query']}

### Instantiate Researcher and Writer Agents

In [67]:
import os
from crewai import Agent, Task, Crew, Process

In [68]:
# Define a new agent for document analysis
document_analyst = Agent(
    role="Document Analyst",
    goal="Analyze documents for historical, cultural, and economic insights",
    backstory="""You are an expert in analyzing various documents and providing comprehensive reports.""",
    llm=chat_llm,
    verbose=True,
    allow_delegation=False,
    tools=[document_analysis_tool],
)

### Define respective tasks

In [69]:
task1 = Task(
    description="""Conduct a comprehensive analysis of the Kingdom of Morocco, focusing on its historical, cultural, and economic aspects as presented in the document 'Morocco-1-3.pdf'.""",
    expected_output="Full analysis report in bullet points",
    agent=document_analyst,
)

task2 = Task(
    description="""Using the insights provided, develop an engaging blog post that highlights the importance of Morocco's historical, cultural, and economic aspects. Your post should be informative yet accessible, catering to a casual audience. Make it sound cool, avoid complex words.""",
    expected_output="Full blog post of at least 4 paragraphs",
    agent=writer,
)

### Instantiate the Crew with a sequential process

In [70]:
crew = Crew(
    agents=[researcher, writer],
    tasks=[task1, task2],
    verbose=2,  # You can set it to 1 or 2 to different logging levels
)

### Initiate the agent workflow: Let the magic begin!

In [71]:
result = crew.kickoff()

print("######################")
print(result)

 [2024-07-29 19:21:19][DEBUG]: == Working Agent: Document Analyst
 [2024-07-29 19:21:19][INFO]: == Starting Task: Conduct a comprehensive analysis of the Kingdom of Morocco, focusing on its historical, cultural, and economic aspects as presented in the document 'Morocco-1-3.pdf'.


> Entering new CrewAgentExecutor chain...
Thought: I need to analyze the document 'Morocco-1-3.pdf' to extract relevant information about the Kingdom of Morocco's historical, cultural, and economic aspects.

Action: Document Analysis Tool
Action Input: {"query": "historical background of Morocco"} 

The region constituting Morocco has been inhabited since the Paleolithic era over 300,000 years ago. The Idrisid dynasty was established by Idris I in 788 and was subsequently ruled by a series of other independent dynasties, reaching its zenith as a regional power in the 11th and 12th centuries, under the Almoravid and Almohad dynasties, when it controlled most of the Iberian Peninsula and the Maghreb. Centuries